In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from cnn_model import run_hyperparameter_search, evaluate_on_test, TimeSeriesDataset, CNNRegressor, train_model, split_LOPO
from torch.utils.data import DataLoader
import neurokit2 as nk  # pip install neurokit2
import seaborn as sns

In [ ]:
from ml_dataloader import process_physiology_data, extract_raw_physio_windows
extract_raw_physio_windows(
    data_path="D:/OneDrive/HubRoom/Projects/SD physiology/Data/processed_individual_anonymized",
    output_path="./ml_processed_raw",
    window_seconds=5,
    stride_seconds=3,
    sampling_rate=120,
)

## Window 2 stride 1

In [1]:
import os
import gc
import json
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

from ml_utils import (
    get_model, evaluate, set_seed,
    run_feature_ablation, grid_search_model, to_loader, to_loader_simple, train_model
)
from ml_pipeline import (
    load_and_split_data, run_feature_ablation,
    select_features_by_ablation, run_grid_search
)
from models import CNNRegressor

# ---------- 설정 ----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pid_array = np.load("ml_processed_raw/pid_array.npy")
unique_pids = np.unique(pid_array)

os.makedirs("results", exist_ok=True)
all_results = {}
for target_pid in unique_pids[4:7]:
    print(f"\n🔁 Processing PID: {target_pid}")

    # ---------- Step 1: 데이터 분리 ----------
    X_train_pre, y_train_pre, _, \
    X_val_pre, y_val_pre, _, \
    X_test, y_test, _, \
    X_finetune, y_finetune, \
    feature_tag_list = load_and_split_data(path="ml_processed_raw", mode="within", target_pid=target_pid)

    dummy_pid_train = np.array(["pretrain"] * len(y_train_pre))
    dummy_pid_val   = np.array(["val"] * len(y_val_pre))

    # ---------- Step 2: Feature Selection ----------
    feature_ablation_df = run_feature_ablation(
        X_train_pre, y_train_pre, dummy_pid_train,
        X_val_pre, y_val_pre, dummy_pid_val,
        feature_tags=feature_tag_list,
        model_type="CNN",
        fixed_params={
            "num_filters": 32,
            "kernel_size": 5,
            "dropout": 0.3,
            "batch_size": 32,
            "learning_rate": 0.001,
            "input_channels": X_train_pre.shape[1]
        },
        num_epochs=5,
        seed=42,
        use_internal_split=False,
        external_val_data=(X_val_pre, y_val_pre)
    )
    selected_features, feature_indices = select_features_by_ablation(
        feature_ablation_df, feature_tag_list
    )

    # ---------- Step 3: Feature 적용 ----------
    X_train_pre = X_train_pre[:, :, feature_indices]
    X_val_pre   = X_val_pre[:, :, feature_indices]

    # ---------- Step 4: Grid Search ----------
    search_space = {
        "num_filters": [32, 64],
        "kernel_size": [3, 5, 7],
        "dropout": [0.2, 0.3, 0.5],
        "batch_size": [32],
        "learning_rate": [0.001, 0.0005, 0.0001]
    }

    best_params, _ = run_grid_search(
        X_train_pre, y_train_pre, dummy_pid_train,
        model_type="CNN",
        search_space=search_space,
        num_epochs=10,
        seed=42,
        use_internal_split=False,
        external_val_data=(X_val_pre, y_val_pre)
    )

    # ---------- Step 5: Pretrain ----------
    X_train_pre = X_train_pre.transpose(0, 2, 1)
    X_val_pre   = X_val_pre.transpose(0, 2, 1)

    pretrained_model, train_losses, val_losses, *_ = train_model(
        X_train_pre, y_train_pre,
        params=best_params,
        model_type="CNN",
        num_epochs=10,
        seed=42,
        pid_array=dummy_pid_train,
        external_val_data=(X_val_pre, y_val_pre),
        use_internal_split=False,
        return_curve=True
    )
    X_finetune = X_finetune.transpose(0, 2, 1)
    X_test     = X_test.transpose(0, 2, 1)
    # ---------- Step 6: Fine-tune ----------
    finetune_model = CNNRegressor(
        input_channels=X_finetune.shape[1],
        num_filters=best_params["num_filters"],
        kernel_size=best_params["kernel_size"],
        dropout=best_params["dropout"]
    ).to(device)

    finetune_optimizer = torch.optim.Adam(finetune_model.parameters(), lr=best_params["learning_rate"])
    criterion = torch.nn.MSELoss()

    
    pretrained_model = pretrained_model.to(device)
    print(f"[{target_pid}] Fine-tune Before → Param sum: {sum(p.sum().item() for p in finetune_model.parameters()):.4f}")
    finetune_model.load_state_dict(pretrained_model.state_dict())
    print(f"[{target_pid}] Fine-tune After  → Param sum: {sum(p.sum().item() for p in finetune_model.parameters()):.4f}")
    del pretrained_model
    torch.cuda.empty_cache()

    

    for epoch in range(5):
        finetune_model.train()
        for i in range(0, len(X_finetune), best_params["batch_size"]):
            X_batch = torch.tensor(X_finetune[i:i+best_params["batch_size"]], dtype=torch.float32).to(device)

            y_batch = torch.tensor(y_finetune[i:i+best_params["batch_size"]], dtype=torch.float32).to(device)
            finetune_optimizer.zero_grad()
            pred = finetune_model(X_batch).squeeze(-1)
            loss = criterion(pred, y_batch)
            loss.backward()
            finetune_optimizer.step()

    # ---------- Step 7: Evaluate ----------
    test_loader = to_loader_simple(
        X_test, y_test,
        batch_size=best_params["batch_size"],
        permute=True,
        shuffle=False
    )

    r2, rmse, mae, _, y_true, y_pred = evaluate(finetune_model, test_loader, device, model_type="CNN")
    print(f"📊 PID {target_pid} | R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")
    print(f"[{target_pid}] y_true mean: {np.mean(y_finetune):.4f}, std: {np.std(y_finetune):.4f}")
    print(f"[{target_pid}] Pretrain loss: {train_losses[-1]:.4f}, Val loss: {val_losses[-1]:.4f}")
    # ---------- Step 8: 저장 ----------
    torch.save(finetune_model.state_dict(), f"results/model_{target_pid}.pt")
    all_results[target_pid] = {
        "r2": r2,
        "rmse": rmse,
        "mae": mae,
        "train_losses": train_losses,
        "val_losses": val_losses,
        "selected_features": feature_indices,
        "best_params": best_params,
        "y_true": y_true,
        "y_pred": y_pred
    }
    with open(f"results/summary_log.txt", "a") as f:
        f.write(f"PID {target_pid} | R2: {r2:.4f} | RMSE: {rmse:.4f} | y_mean: {np.mean(y_finetune):.4f} | y_std: {np.std(y_finetune):.4f}\n")
    with open(f"results/metrics_{target_pid}.json", "w") as f:
        json.dump({
            "pid": target_pid,
            "r2": float(r2),
            "rmse": float(rmse),
            "mae": float(mae)
        }, f, indent=2)
    np.save(f"results/y_pred_{target_pid}.npy", y_pred)
    np.save(f"results/train_losses_{target_pid}.npy", np.array(train_losses))
    np.save(f"results/val_losses_{target_pid}.npy", np.array(val_losses))

    with open(f"results/features_{target_pid}.json", "w") as f:
        json.dump({
            "selected_features": feature_indices
        }, f, indent=2)

    with open(f"results/params_{target_pid}.json", "w") as f:
        json.dump(best_params, f, indent=2)
    torch.cuda.empty_cache()
    gc.collect()
    import pandas as pd
    df_summary = pd.DataFrame.from_dict(all_results, orient="index")
    df_summary.to_csv("results/summary_all.csv")



🔁 Processing PID: 005
🧪 Training baseline | use_internal_split = False


c:\Users\user\anaconda3\envs\ml_env\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\user\anaconda3\envs\ml_env\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\user\anaconda3\envs\ml_env\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  

🔍 Removing EDA_Tonic (1/11)
🧪 Training EDA_Tonic | use_internal_split = False


c:\Users\user\anaconda3\envs\ml_env\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\user\anaconda3\envs\ml_env\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\user\anaconda3\envs\ml_env\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  

🔍 Removing EDA_Phasic (2/11)
🧪 Training EDA_Phasic | use_internal_split = False


...

c:\Users\user\anaconda3\envs\ml_env\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\user\anaconda3\envs\ml_env\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\user\anaconda3\envs\ml_env\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  


🏆 Best Hyperparameters:
num_filters : 32
kernel_size : 7
dropout     : 0.2
batch_size  : 32
learning_rate: 0.0001
input_channels: 120
✅ Grid Search 완료!


c:\Users\user\anaconda3\envs\ml_env\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\user\anaconda3\envs\ml_env\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\user\anaconda3\envs\ml_env\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  

[007] Fine-tune Before → Param sum: 1.6086
[007] Fine-tune After  → Param sum: -2.3355
📊 PID 007 | R²: -11.1479, RMSE: 1.5682, MAE: 1.5023
[007] y_true mean: 0.8643, std: 0.4963
[007] Pretrain loss: 0.6012, Val loss: 0.6010


c:\Users\user\anaconda3\envs\ml_env\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


## Window 5 stride 3

In [ ]:
import os
import gc
import json
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

from ml_utils import (
    get_model, evaluate, set_seed,
    run_feature_ablation, grid_search_model, to_loader, to_loader_simple, train_model
)
from ml_pipeline import (
    load_and_split_data, run_feature_ablation,
    select_features_by_ablation, run_grid_search
)
from models import CNNRegressor

# ---------- 설정 ----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pid_array = np.load("ml_processed_raw/pid_array.npy")
unique_pids = np.unique(pid_array)

os.makedirs("results", exist_ok=True)
all_results = {}
for target_pid in unique_pids[4:7]:
    print(f"\n🔁 Processing PID: {target_pid}")

    # ---------- Step 1: 데이터 분리 ----------
    X_train_pre, y_train_pre, _, \
    X_val_pre, y_val_pre, _, \
    X_test, y_test, _, \
    X_finetune, y_finetune, \
    feature_tag_list = load_and_split_data(path="ml_processed_raw", mode="within", target_pid=target_pid)

    dummy_pid_train = np.array(["pretrain"] * len(y_train_pre))
    dummy_pid_val   = np.array(["val"] * len(y_val_pre))

    # ---------- Step 2: Feature Selection ----------
    feature_ablation_df = run_feature_ablation(
        X_train_pre, y_train_pre, dummy_pid_train,
        X_val_pre, y_val_pre, dummy_pid_val,
        feature_tags=feature_tag_list,
        model_type="CNN",
        fixed_params={
            "num_filters": 32,
            "kernel_size": 5,
            "dropout": 0.3,
            "batch_size": 32,
            "learning_rate": 0.001,
            "input_channels": X_train_pre.shape[1]
        },
        num_epochs=5,
        seed=42,
        use_internal_split=False,
        external_val_data=(X_val_pre, y_val_pre)
    )
    selected_features, feature_indices = select_features_by_ablation(
        feature_ablation_df, feature_tag_list
    )

    # ---------- Step 3: Feature 적용 ----------
    X_train_pre = X_train_pre[:, :, feature_indices]
    X_val_pre   = X_val_pre[:, :, feature_indices]

    # ---------- Step 4: Grid Search ----------
    search_space = {
        "num_filters": [32, 64],
        "kernel_size": [3, 5, 7],
        "dropout": [0.2, 0.3, 0.5],
        "batch_size": [32],
        "learning_rate": [0.001, 0.0005, 0.0001]
    }

    best_params, _ = run_grid_search(
        X_train_pre, y_train_pre, dummy_pid_train,
        model_type="CNN",
        search_space=search_space,
        num_epochs=10,
        seed=42,
        use_internal_split=False,
        external_val_data=(X_val_pre, y_val_pre)
    )

    # ---------- Step 5: Pretrain ----------
    X_train_pre = X_train_pre.transpose(0, 2, 1)
    X_val_pre   = X_val_pre.transpose(0, 2, 1)

    pretrained_model, train_losses, val_losses, *_ = train_model(
        X_train_pre, y_train_pre,
        params=best_params,
        model_type="CNN",
        num_epochs=10,
        seed=42,
        pid_array=dummy_pid_train,
        external_val_data=(X_val_pre, y_val_pre),
        use_internal_split=False,
        return_curve=True
    )
    X_finetune = X_finetune.transpose(0, 2, 1)
    X_test     = X_test.transpose(0, 2, 1)
    # ---------- Step 6: Fine-tune ----------
    finetune_model = CNNRegressor(
        input_channels=X_finetune.shape[1],
        num_filters=best_params["num_filters"],
        kernel_size=best_params["kernel_size"],
        dropout=best_params["dropout"]
    ).to(device)

    finetune_optimizer = torch.optim.Adam(finetune_model.parameters(), lr=best_params["learning_rate"])
    criterion = torch.nn.MSELoss()

    
    pretrained_model = pretrained_model.to(device)
    print(f"[{target_pid}] Fine-tune Before → Param sum: {sum(p.sum().item() for p in finetune_model.parameters()):.4f}")
    finetune_model.load_state_dict(pretrained_model.state_dict())
    print(f"[{target_pid}] Fine-tune After  → Param sum: {sum(p.sum().item() for p in finetune_model.parameters()):.4f}")
    del pretrained_model
    torch.cuda.empty_cache()

    

    for epoch in range(5):
        finetune_model.train()
        for i in range(0, len(X_finetune), best_params["batch_size"]):
            X_batch = torch.tensor(X_finetune[i:i+best_params["batch_size"]], dtype=torch.float32).to(device)

            y_batch = torch.tensor(y_finetune[i:i+best_params["batch_size"]], dtype=torch.float32).to(device)
            finetune_optimizer.zero_grad()
            pred = finetune_model(X_batch).squeeze(-1)
            loss = criterion(pred, y_batch)
            loss.backward()
            finetune_optimizer.step()

    # ---------- Step 7: Evaluate ----------
    test_loader = to_loader_simple(
        X_test, y_test,
        batch_size=best_params["batch_size"],
        permute=True,
        shuffle=False
    )

    r2, rmse, mae, _, y_true, y_pred = evaluate(finetune_model, test_loader, device, model_type="CNN")
    print(f"📊 PID {target_pid} | R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")
    print(f"[{target_pid}] y_true mean: {np.mean(y_finetune):.4f}, std: {np.std(y_finetune):.4f}")
    print(f"[{target_pid}] Pretrain loss: {train_losses[-1]:.4f}, Val loss: {val_losses[-1]:.4f}")
    # ---------- Step 8: 저장 ----------
    torch.save(finetune_model.state_dict(), f"results/model_{target_pid}.pt")
    all_results[target_pid] = {
        "r2": r2,
        "rmse": rmse,
        "mae": mae,
        "train_losses": train_losses,
        "val_losses": val_losses,
        "selected_features": feature_indices,
        "best_params": best_params,
        "y_true": y_true,
        "y_pred": y_pred
    }
    with open(f"results/summary_log.txt", "a") as f:
        f.write(f"PID {target_pid} | R2: {r2:.4f} | RMSE: {rmse:.4f} | y_mean: {np.mean(y_finetune):.4f} | y_std: {np.std(y_finetune):.4f}\n")
    with open(f"results/metrics_{target_pid}.json", "w") as f:
        json.dump({
            "pid": target_pid,
            "r2": float(r2),
            "rmse": float(rmse),
            "mae": float(mae)
        }, f, indent=2)
    np.save(f"results/y_pred_{target_pid}.npy", y_pred)
    np.save(f"results/train_losses_{target_pid}.npy", np.array(train_losses))
    np.save(f"results/val_losses_{target_pid}.npy", np.array(val_losses))

    with open(f"results/features_{target_pid}.json", "w") as f:
        json.dump({
            "selected_features": feature_indices
        }, f, indent=2)

    with open(f"results/params_{target_pid}.json", "w") as f:
        json.dump(best_params, f, indent=2)
    torch.cuda.empty_cache()
    gc.collect()
    import pandas as pd
    df_summary = pd.DataFrame.from_dict(all_results, orient="index")
    df_summary.to_csv("results/summary_all.csv")



🔁 Processing PID: 005
🧪 Training baseline | use_internal_split = False
🔍 Removing EDA_Tonic (1/11)
🧪 Training EDA_Tonic | use_internal_split = False
🔍 Removing EDA_Phasic (2/11)
🧪 Training EDA_Phasic | use_internal_split = False
🔍 Removing SCR_Amplitude (3/11)
🧪 Training SCR_Amplitude | use_internal_split = False
🔍 Removing SCR_RiseTime (4/11)
🧪 Training SCR_RiseTime | use_internal_split = False
🔍 Removing PPG_Rate (5/11)
🧪 Training PPG_Rate | use_internal_split = False
🔍 Removing RSP_Rate (6/11)
🧪 Training RSP_Rate | use_internal_split = False
🔍 Removing RSP_RVT (7/11)
🧪 Training RSP_RVT | use_internal_split = False
🔍 Removing RSP_Amplitude (8/11)
🧪 Training RSP_Amplitude | use_internal_split = False
🔍 Removing pupilL (9/11)
🧪 Training pupilL | use_internal_split = False
🔍 Removing pupilR (10/11)
🧪 Training pupilR | use_internal_split = False
🔍 Removing pupil_mean (11/11)
🧪 Training pupil_mean | use_internal_split = False
📌 선택된 feature 수: 11 / 11
📌 feature_indices: [0, 5, 4, 7, 6, 3,

In [12]:
# ---------- Step 7: Evaluate ----------
test_loader = to_loader_simple(
    X_test, y_test,
    batch_size=best_params["batch_size"],
    permute=True,
    shuffle=False
)

r2, rmse, mae, _, y_true, y_pred = evaluate(finetune_model, test_loader, device, model_type="CNN")
print(f"📊 PID {target_pid} | R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")

# ---------- Step 8: 저장 ----------
torch.save(finetune_model.state_dict(), f"results/model_{target_pid}.pt")
all_results[target_pid] = {
    "r2": r2,
    "rmse": rmse,
    "mae": mae,
    "train_losses": train_losses,
    "val_losses": val_losses,
    "selected_features": feature_indices,
    "best_params": best_params,
    "y_true": y_true,
    "y_pred": y_pred
}
with open(f"results/metrics_{target_pid}.json", "w") as f:
    json.dump({
        "pid": target_pid,
        "r2": float(r2),
        "rmse": float(rmse),
        "mae": float(mae)
    }, f, indent=2)
np.save(f"results/y_pred_{target_pid}.npy", y_pred)
np.save(f"results/train_losses_{target_pid}.npy", np.array(train_losses))
np.save(f"results/val_losses_{target_pid}.npy", np.array(val_losses))

with open(f"results/features_{target_pid}.json", "w") as f:
    json.dump({
        "selected_features": feature_indices
    }, f, indent=2)

with open(f"results/params_{target_pid}.json", "w") as f:
    json.dump(best_params, f, indent=2)
torch.cuda.empty_cache()
gc.collect()

📊 PID 001 | R²: -0.2787, RMSE: 1.6097, MAE: 1.3707


1993

📊 PID 001 | R²: -0.8620, RMSE: 1.9425, MAE: 1.6846


2518